<a href="https://colab.research.google.com/github/petewarden/pico_colabs/blob/main/Building_Person_Detection_for_the_Arducam_Pico4ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Building Person Detection for the Arducam Pico4ML

*By [Pete Warden](https://twitter.com/petewarden), peteward@stanford.edu*

This notebook demonstrates how to compile and deploy some programs for [the Arducam Pico4ML board](), which is based on a Raspberry Pi Pico RP2040 microcontroller, but adds sensors and a screen to make it easier to use for TinyML development. A lot of the setup steps are the same as for [the blink Pico example](https://colab.research.google.com/github/petewarden/pico_colabs/blob/main/Building_Blink_for_the_Raspberry_Pi_Pico.ipynb) so I don't go into as much detail on the basics.

## Install the SDK and Examples

This is similar to the blink notebook, but instead of installing the example code for the standalone Pico board, we fetch [the Arducam repository of sample code]( https://github.com/ArduCAM/pico-tflmicro).

In [1]:
!mkdir pico
%cd pico
!git clone -b master https://github.com/raspberrypi/pico-sdk.git
%cd pico-sdk
!git submodule update --init
%cd ..
!git clone --recursive  https://github.com/ArduCAM/pico-tflmicro

/content/pico
Cloning into 'pico-sdk'...
remote: Enumerating objects: 5518, done.
remote: Counting objects: 100% (42/42), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 5518 (delta 19), reused 24 (delta 11), pack-reused 5476
Receiving objects: 100% (5518/5518), 2.49 MiB | 7.09 MiB/s, done.
Resolving deltas: 100% (2807/2807), done.
/content/pico/pico-sdk
Submodule 'lib/cyw43-driver' (https://github.com/georgerobotics/cyw43-driver.git) registered for path 'lib/cyw43-driver'
Submodule 'lib/lwip' (https://github.com/lwip-tcpip/lwip.git) registered for path 'lib/lwip'
Submodule 'tinyusb' (https://github.com/hathach/tinyusb.git) registered for path 'lib/tinyusb'
Cloning into '/content/pico/pico-sdk/lib/cyw43-driver'...
Cloning into '/content/pico/pico-sdk/lib/lwip'...
Cloning into '/content/pico/pico-sdk/lib/tinyusb'...
Submodule path 'lib/cyw43-driver': checked out '195dfcc10bb6f379e3dea45147590db2203d3c7b'
Submodule path 'lib/lwip': checked out '239918ccc173cb2c2a62f4

## Install the Toolchain

In [2]:
!sudo apt update
!sudo apt install cmake gcc-arm-none-eabi libnewlib-arm-none-eabi build-essential libstdc++-arm-none-eabi-newlib

Ign:1 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:7 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [92.1 kB]
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:10 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:12 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 Packages [3,397 kB]
Hit:13 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:14

## Set up the Build System for the Video Streaming Example

The first program we're going to build is an example that takes the output from the builtin camera on the Pico4ML board and copies it to the screen. This is simpler than the person detection example, so it's a good way to make sure we have the system working well.



## Set up the Person Detection Build Files

Here we're going to configure the person detection example, in a similar way to how we set up the blink build files. Because the folder structure of the Arducam repository is a little different from the Raspberry Pi examples, we have to set the `PICO_SDK_PATH` a bit differently though.

In [3]:
%env PICO_SDK_PATH=/content/pico/pico-sdk
%cd /content/pico/pico-tflmicro
!mkdir build 
%cd build 
!cmake ..

env: PICO_SDK_PATH=/content/pico/pico-sdk
/content/pico/pico-tflmicro
/content/pico/pico-tflmicro/build
Using PICO_SDK_PATH from environment ('/content/pico/pico-sdk')
PICO_SDK_PATH is /content/pico/pico-sdk
Defaulting PICO_PLATFORM to rp2040 since not specified.
Defaulting PICO platform compiler to pico_arm_gcc since not specified.
-- Defaulting build type to 'Release' since not specified.
PICO compiler is pico_arm_gcc
-- The C compiler identification is GNU 6.3.1
-- The CXX compiler identification is GNU 6.3.1
-- The ASM compiler identification is GNU
-- Found assembler: /usr/bin/arm-none-eabi-gcc
Build type is Release
Defaulting PICO target board to pico since not specified.
Using board configuration from /content/pico/pico-sdk/src/boards/include/boards/pico.h
-- Found Python3: /usr/bin/python3.7 (found version "3.7.14") found components: Interpreter 
TinyUSB available at /content/pico/pico-sdk/lib/tinyusb/src/portable/raspberrypi/rp2040; enabling build support for USB.
cyw43-driver

## Compile the Person Detection Example

The person detector requires [the TensorFlow Lite Micro library](https://github.com/tensorflow/tflite-micro) to run machine learning models, and this is quite a large framework so the compilation process may take ten minutes or so. It can also use a lot of RAM, so to prevent the Colab runtime from crashing we'll only compile a few files in parallel, with the `-j4` option to `make`.

In [4]:
%cd /content/pico/pico-tflmicro/build
!make -j4

/content/pico/pico-tflmicro/build
Scanning dependencies of target bs2_default
[  0%] Creating directories for 'ELF2UF2Build'
[  0%] Creating directories for 'PioasmBuild'
[  0%] Building ASM object pico-sdk/src/rp2_common/boot_stage2/CMakeFiles/bs2_default.dir/compile_time_choice.S.obj
[  0%] Linking ASM executable bs2_default.elf
[  0%] No download step for 'ELF2UF2Build'
[  0%] No download step for 'PioasmBuild'
[  0%] Built target bs2_default
[  0%] No update step for 'PioasmBuild'
[  0%] No update step for 'ELF2UF2Build'
[  0%] No patch step for 'PioasmBuild'
[  0%] Generating bs2_default.bin
[  0%] No patch step for 'ELF2UF2Build'
[  0%] Performing configure step for 'PioasmBuild'
[  0%] Generating bs2_default_padded_checksummed.S
loading initial cache file /content/pico/pico-tflmicro/build/pico-sdk/src/rp2_common/cyw43_driver/pioasm/tmp/PioasmBuild-cache-Release.cmake
[  0%] Performing configure step for 'ELF2UF2Build'
[  0%] Built target bs2_default_padded_checksummed_asm
Scanni

## Load and Run the Person Detector Executable

You should now be able to find the video streaming binary in `/content/pico/RPI-Pico-Cam/arducam_demo/build/arducam/arducam_demo.uf2`. Press the `bootsel` button on the board, plug it into the USB socket so that the mass storage drive appears in the file system, and copy the binary over. After the board reboots, you should see the output of the camera in the board's display, together with a percentage score at the bottom. If you hold the board with the USB connector at the bottom, and point the camera to a person, you should see the percentage rise from a low value to much higher, indicating that the person detection model is working.